#Nokia_People_and_Planet_Report_2020.pdf

In [1]:
!nvidia-smi

Sun Sep 24 19:45:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq langchain==0.0.266 --progress-bar off
!pip install -qqq chromadb==0.4.5 --progress-bar off
!pip install -qqq pypdf==3.15.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq InstructorEmbedding==1.0.1 --progress-bar off
!pip install -qqq pdf2image==1.16.3 --progress-bar off

In [3]:
!wget -q https://github.com/PanQiWei/AutoGPTQ/releases/download/v0.4.1/auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl

In [4]:
!pip install -qqq auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl --progress-bar off

In [5]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [6]:
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path
from transformers import AutoTokenizer, TextStreamer, pipeline

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

## Data

In [7]:
!mkdir pdfs

mkdir: cannot create directory ‘pdfs’: File exists


In [8]:
!rm -rf "db"

In [9]:
loader = PyPDFDirectoryLoader("/content/pdfs")
docs = loader.load()
len(docs)

102

In [10]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
)

load INSTRUCTOR_Transformer
max_seq_length  512


In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

402

In [12]:
%%time
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

CPU times: user 30.6 s, sys: 1.5 s, total: 32.1 s
Wall time: 42.4 s


## Llama 2 13B

In [13]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)

In [14]:
!nvidia-smi

Sun Sep 24 19:48:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    35W /  70W |   9647MiB / 15360MiB |     92%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [15]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <<SYS>>
{system_prompt}
<</SYS>>

{prompt} [/INST]
""".strip()

In [16]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [17]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausal

In [18]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

In [19]:
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

In [20]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [21]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

## Chat with Multiple PDFs

#Shell Plc

In [22]:
result = qa_chain("What  is direct GHG Emissions scope 1  in 2020 ")



 Based on the information provided, the answer to your question is:

Direct GHG Emissions (Scope 1) in 2020 = 12.8 million tonnes CO2 equivalent.

This can be found in the table labeled "SCOPE 1 GHG EMISSIONS (OPERATIONAL CONTROL)" under the column for "2020" and the row for "Direct GHG Emissions (Scope 1)".


In [23]:
result = qa_chain("What Scope 2GHG Emissions location- based method in 2020  ")



 Based on the information provided, the answer to your question is:

Scope 2 GHG Emissions (location-based method) in 2020 = 11 million tonnes CO2e.


In [ ]:
result = qa_chain("What Scope 2GHG Emissions ,market- based in 2020  metric tons CO2e")



 Based on the information provided in the table, the Scope 2 GHG emissions for the market-based method in 2020 were 9 million tonnes CO2e.


In [26]:
result = qa_chain("What Scope 3GHG Emissions in metric tons CO2e in 2020 ")


 Based on the information provided, the answer to your question is:

Scope 3 GHG Emissions in 2020 = 3 + 3 + 3 + 3 + 3 = 15 million tonnes CO2e.

Explanation:

The table provides data for Scope 1, 2, and 3 GHG emissions for the year 2020. Scope 3 emissions are defined as "Emissions related to activities from assets not owned or controlled by the company, but that the company has influence over."

In this case, the Scope 3 emissions include:

* Exported energy (3 million tonnes CO2e)
* Upstream emissions (3 million tonnes CO2e)
* Integrated Gas emissions (3 million tonnes CO2e)
* Downstream emissions (3 million tonnes CO2e)

Therefore, the total Scope 3 GHG emissions in 2020 were 3 + 3 + 3 + 3 = 15 million tonnes CO2e.


In [ ]:
result = qa_chain(" What were the estimated Scope 3 GHG emissions from owned production in 2020?")

 Based on the information provided in the table, the estimated Scope 3 GHG emissions from owned production in 2020 were 1.9 million tonnes of CO2 equivalents.


In [ ]:
result = qa_chain("What was the total amount of non-hazardous waste disposed of in 2020? ")



 Based on the information provided, the total amount of non-hazardous waste disposed of in 2020 was 1,465 thousand tonnes. This information can be found in the second line of the table labeled "Non-hazardous waste disposed" in the text.


In [ ]:
result = qa_chain("What was the total amount of hazardous waste disposed of in 2020? ")



 Based on the information provided in the text, the total amount of hazardous waste disposed of in 2020 was 555 thousand tonnes. This can be found in the table labeled "Hazardous waste disposed (thousand tonnes)" under the section titled "WASTE MANAGEMENT."


In [ ]:
result = qa_chain(" What was the total amount of waste beneficially reused, recycled, or recovered in 2020? ")



 Based on the information provided in the text, the total amount of waste beneficially reused, recycled, or recovered in 2020 was 465 thousand tonnes. This information can be found in the table labeled "Waste beneficially reused, recycled, or recovered" under section [A].


In [ ]:
result = qa_chain(" What was the Total  waste disposed in 2020? ")



 Based on the information provided in the text, the total waste disposed in 2020 was:

Total waste disposed = Hazardous waste disposed + Non-hazardous waste disposed
= 555 + 1,465
= 2,020 (thousand tonnes)

Therefore, the answer is 2,020 thousand tonnes.


In [ ]:
result = qa_chain(" what is   water usage and discharge  in 2020 ? ")



 Based on the information provided in the text, I can answer the following questions:

What was the total water withdrawal in 2020?
Answer: The total water withdrawal in 2020 was 171 million cubic meters.

What was the total water consumption in 2020?
Answer: The total water consumption in 2020 was 127 million cubic meters.

What was the total water discharge in 2020?
I cannot answer this question based on the information provided because there is no data given for water discharge in 2020.


In [ ]:
result = qa_chain(" What was the total water consumption in 2020?")



 Based on the information provided, the total water consumption in 2020 was 127 million cubic meters. This can be found in the "Fresh water consumed" column under the "Water Use and Discharge" section.


In [ ]:
result = qa_chain("What was the total water returned in 2020?")





 Based on the information provided in the table, the total water returned in 2020 was 45 million cubic meters. This can be found in the "Fresh water returned" column under the "Million cubic meters" heading.


In [ ]:
result = qa_chain(" what Total energy usage  in 2020in Energy usage table ? ")

 Based on the information provided in the table, the total energy usage in 2020 was 269 million megawatt-hours (MWh). This is calculated by adding up the imported electricity, imported steam and heat, exported electricity, and exported steam and heat, and then subtracting the renewable sources - purchased electricity, renewable sources - purchased steam, and renewable sources - electricity exported to grid.


In [ ]:
result = qa_chain("What  is direct GHG Emissions scope 1  in 2020 ")



 Based on the information provided in the table, the direct GHG Emissions scope 1 in 2020 was 63 million tonnes of CO2 equivalent.


In [ ]:
result = qa_chain("What Scope 2GHG Emissions location- based method in 2020  ")



 Based on the information provided, the answer to your question is:

Scope 2 GHG Emissions (location-based method) in 2020 = 11 million tonnes CO2e.


In [ ]:
result = qa_chain("What Scope 2GHG Emissions ,market- based in 2020  metric tons CO2e")



 Based on the information provided in the table, the Scope 2 GHG emissions for the market-based method in 2020 were 9 million tonnes CO2e.


In [ ]:
result = qa_chain("What Scope 3GHG Emissions in 2020 ")


 Based on the information provided in the table, the answer to the question "What Scope 3 GHG emissions in 2020?" would be:

"We do not have the 2020 data yet, as it will be available in June 2021."

The table only provides information up to 2019, and there is no mention of 2020 data. Therefore, the answer cannot be determined based on the given information.


In [ ]:
result = qa_chain(" What were the estimated Scope 3 GHG emissions from owned production in 2020?")

 Based on the information provided in the table, the estimated Scope 3 GHG emissions from owned production in 2020 were 1.9 million tonnes of CO2 equivalents.


In [ ]:
result = qa_chain("What was the total hazardous waste in 2020? ")



 Based on the information provided in the text, I can answer your question.

According to the table labeled "ENV-7 – 306-3", the total hazardous waste disposed of in 2020 was 698 thousand tonnes.


In [ ]:
result = qa_chain("What was the total amount of hazardous waste disposed of in 2020? ")



 Based on the information provided in the text, the total amount of hazardous waste disposed of in 2020 was 555 thousand tonnes. This can be found in the table labeled "Hazardous waste disposed (thousand tonnes)" under the section titled "WASTE MANAGEMENT."


In [ ]:
result = qa_chain(" What was the total amount of waste beneficially reused, recycled, or recovered in 2020? ")



 Based on the information provided in the text, the total amount of waste beneficially reused, recycled, or recovered in 2020 was 465 thousand tonnes. This information can be found in the table labeled "Waste beneficially reused, recycled, or recovered" under section [A].


In [ ]:
result = qa_chain(" What was the Total  waste disposed in 2020? ")



 Based on the information provided in the text, the total waste disposed in 2020 was:

Total waste disposed = Hazardous waste disposed + Non-hazardous waste disposed
= 555 + 1,465
= 2,020 (thousand tonnes)

Therefore, the answer is 2,020 thousand tonnes.


In [ ]:
result = qa_chain(" what is   water usage and discharge  in 2020 ? ")



 Based on the information provided in the text, I can answer the following questions:

What was the total water withdrawal in 2020?
Answer: The total water withdrawal in 2020 was 171 million cubic meters.

What was the total water consumption in 2020?
Answer: The total water consumption in 2020 was 127 million cubic meters.

What was the total water discharge in 2020?
I cannot answer this question based on the information provided because there is no data given for water discharge in 2020.


In [ ]:
result = qa_chain(" What was the total water consumption in 2020?")



 Based on the information provided, the total water consumption in 2020 was 127 million cubic meters. This can be found in the "Fresh water consumed" column under the "Water Use and Discharge" section.


In [ ]:
result = qa_chain("What was the total water returned in 2020?")





 Based on the information provided in the table, the total water returned in 2020 was 45 million cubic meters. This can be found in the "Fresh water returned" column under the "Million cubic meters" heading.


In [ ]:
result = qa_chain(" what Total Total volume of waste disposed of in 2020   ? ")

 Based on the information provided, the total volume of waste disposed of in 2020 was 2,020 thousand tonnes.


In [24]:
result = qa_chain(" what isTotal number of  Employees  in 2020 ? ")

 Based on the information provided, the total number of employees in 2020 was 87,000.


#SMSUNG

In [25]:
result = qa_chain("What Direct GHG emissions Scope 1  in 2020 ?")




 Based on the information provided, the direct GHG emissions (Scope 1) in 2020 were 5,726 thousand tonnes of CO2e.


In [28]:
result = qa_chain("What  Scope 2 location -based in tons in 2020 ?")




 Based on the information provided, I can answer your question.

According to the table, the Scope 2 location-based emissions in tons for 2020 are:

* Suppliers: 8,030 thousand tonnes CO2e
* Logistics: 6,682 thousand tonnes CO2e

So, the total Scope 2 location-based emissions in 2020 were 14,692 thousand tonnes CO2e.


In [29]:
result = qa_chain("What  Scope 2 market -based in tons in 2020 ?")

 Based on the information provided, I can answer your question.

The Scope 2 market-based emissions in tons for 2020 are not explicitly stated in the given context. However, we can calculate the total Scope 2 emissions based on the information provided.

According to the table, the indirect GHG emissions (Scope 2) in 2020 were 10,266 thousand tonnes CO2e. This includes emissions from purchased energy (PFCs, HFCs), employee travel, and waste disposal.

Therefore, the answer to your question is:

Scope 2 market-based emissions in tons for 2020 = 10,266 thousand tonnes CO2e.


In [24]:
result = qa_chain("What   GHG emissions from other sources Scope 3  green house table   supplies thousand tonnes CO2e in 2020")




 Based on the information provided in the table, the GHG emissions from other sources (Scope 3) for supplies in 2020 were 8,030 thousand tonnes CO2e.


In [ ]:
result = qa_chain("What   GHG emissions from other sources Scope 3  green house table   employee bussiness thousand tonnes CO2e in 2020")




 Based on the information provided in the table, the GHG emissions from other sources (Scope 3) for employees' business trips in 2020 were 14 thousand tonnes CO2e.


In [ ]:
result = qa_chain("What   GHG emissions from other sources Scope 3  green house table   logistics  ")



 Based on the information provided, here are the answers to your questions:

1. GHG emissions from other sources (Scope 3):
The table shows that the company had GHG emissions from other sources (Scope 3) of 7,952, 8,278, and 8,030 thousand tonnes CO2e in 2018, 2019, and 2020, respectively.
2. Logistics GHG emissions:
The table shows that the company had logistics GHG emissions of 7,846, 8,223, and 6,682 thousand tonnes CO2e in 2018, 2019, and 2020, respectively.
3. GHG emissions from business trips:
The table shows that the company had GHG emissions from business trips of 110, 106, and 14 thousand tonnes CO2e in 2018, 2019, and 2020, respectively.


In [30]:
result = qa_chain(" what isTotal hazardous waste  produced in tons in 2020? ")

 Based on the information provided, I can answer your question.

The total hazardous waste produced in tons in 2020 was 345,866 tons. This is calculated by adding up the amounts of hazardous waste produced in each year:

396,690 + 321,627 + 345,866 = 345,866

Please note that this answer is based on the information provided and may not be accurate if other factors are taken into account.


In [31]:
result = qa_chain("What Total amount of   waste of produced in tons  in 2020 ? ")

 Based on the information provided, the total amount of waste produced in 2020 was 1,181,741 tons. This includes general waste (835,875 tons), hazardous waste (345,866 tons), and wastewater discharge (109,201 tons).


In [32]:
result = qa_chain(" what isTotal number of  Employees  in 2020 ? ")

 Based on the information provided, I can answer the following questions:

1. What was the total number of employees in 2020?

According to the information provided, the total number of employees in 2020 was 137,365. This includes both direct employees and contractors.

2. What was the turnover rate for global employees excluding South Korea in 2020?

The turnover rate for global employees excluding South Korea in 2020 was 17.9%.

3. What was the number of contractors and apprentices in South Korea in 2020?

The number of contractors and apprentices in South Korea in 2020 was 2,552.

I do not have the answers to the following questions:

4. What was the total number of employees in 2018 and 2019?

The information provided does not include the total number of employees in 2018 and 2019.

5. What was the waste management performance in 2020?

The information provided does not include the waste management performance in 2020.

6. What was the water management performance in 2020?

The inf

In [ ]:
result = qa_chain(" what is    total water usage industerial water in thousand tonnes")




 Based on the information provided in the sustainability report, the total industrial water usage for Samsung Electronics in 2020 was 75,243 thousand tonnes. This is calculated by adding the tap water usage (54,434 thousand tonnes) and underground water usage (959 thousand tonnes), and then subtracting the reused water (62,371 thousand tonnes) and ultra-pure water supply amount (57,226 thousand tonnes).

Therefore, the answer to your question is:

Total industrial water usage = Tap water usage + Underground water usage - Reused water - Ultra-pure water supply amount
= 54,434 + 959 - 62,371 - 57,226
= 75,243 thousand tonnes


In [ ]:
result = qa_chain(" what is    total water usage underground water in thousand tonnes")




 Based on the information provided in the sustainability report, the total water usage, including underground water, can be calculated as follows:

Total water usage = Surface water withdrawal + Underground water withdrawal

According to the table, the surface water withdrawal was 142,294 thousand tonnes, and the underground water withdrawal was 585 thousand tonnes. Therefore, the total water usage is:

Total water usage = 142,294 + 585 = 142,879 thousand tonnes

So, the answer is:

Total water usage underground water in thousand tonnes = 585 thousand tonnes.


In [ ]:
result = qa_chain(" what is total  water usage   in 2020 ? ")

 Based on the information provided, the total water usage in 2020 is not explicitly stated. However, we can calculate it by adding the industrial water usage, tap water usage, and underground water usage.

Industrial water usage in 2020: 78,837 + 81,984 + 75,243 = 235,064 thousand tonnes

Tap water usage in 2020: 54,434 + 51,839 + 66,466 = 172,739 thousand tonnes

Underground water usage in 2020: 959 + 657 + 585 = 2,201 thousand tonnes

Total water usage in 2020: 235,064 + 172,739 + 2,201 = 410,004 thousand tonnes

Therefore, the total water usage in 2020 is 410,004 thousand tonnes.


In [ ]:
result = qa_chain("what is  Employee benefit expenses  in 2020 KRW billion?")






 Based on the information provided, I can answer your question.

Employee benefit expenses in 2020 were KRW 4,655 billion.


In [ ]:
result = qa_chain("What is Percentage of female employees1) in 2020?")







 Based on the information provided, the percentage of female employees in 2020 was 37.3%.


In [ ]:
result = qa_chain("What was the percentage of female employees by job title in 2020?")








 Based on the information provided, I can answer the following questions:

1. What was the percentage of female employees by job title in 2020?

According to the table, the percentage of female employees by job title in 2020 was:

* Staff: 46.3%
* Managers: 15.3%
* Executives: 6.6%

So, the percentage of female employees by job title in 2020 was:

46.3% + 15.3% + 6.6% = 68.2%

Therefore, the answer is: 68.2%.


#Microsoft

In [ ]:

result = qa_chain("What is Scope 1– CO2 Greenhouse gas emissions (mtCO2e) FY20?")













 Based on the information provided in the appendix, the answer to the question "What is Scope 1– CO2 Greenhouse gas emissions (mtCO2e) FY20?" is:

Scope 1– CO2 Greenhouse gas emissions (mtCO2e) FY20 = 96,700.

This information can be found in Table 7 of the appendix, under the heading "Greenhouse Gas Emissions (mtCO2e)".


In [ ]:
result = qa_chain("What is Scope 2 – Location-based²  Greenhouse gas emissions (mtCO2e)  FY20? ")

 Based on the information provided in the appendix, I can answer the question.

According to the table in Appendix B, the Scope 2 – Location-based – Greenhouse gas emissions (mtCO2e) for FY20 are:

Europe, Middle East, Africa: 860,858
Asia: 804,567
Latin America: 3,871
North America: 43,859

Therefore, the total Scope 2 – Location-based – Greenhouse gas emissions (mtCO2e) for FY20 are:

860,858 + 804,567 + 3,871 + 43,859 = 1,792,255 mtCO2e.


In [ ]:
result = qa_chain("What is Scope 2 – Market-based Greenhouse gas emissions (mtCO2e)FY20?")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Based on the information provided, I cannot determine what Scope 2 - Market-based Greenhouse gas emissions (mtCO2e) FY20 is. The information provided does not include any data or information about Scope 2 emissions for FY20.


In [ ]:
result = qa_chain("What is Scope 3 – GHG emissions from all categories FY20? ")


 Based on the information provided, I can answer the question.

Scope 3 – GHG emissions from all categories FY20 = 1,870,000.

This figure is calculated by adding up all the emissions from the different Scope 3 categories listed in the table:

Scope 3 – Fuel- and energy-related activities (Location-based): 550,000 + 650,000 = 1,200,000
Scope 3 – Fuel- and energy-related activities (Market-based): 89,000 + 110,000 = 199,000
Scope 3 – Upstream transportation: 50,000 + 100,000 = 150,000
Scope 3 – Waste: 700 + 1,100 = 1,800
Scope 3 – Business travel: 419,020 + 461,787 = 880,807
Scope 3 – Employee commuting: 343,000 + 345,000 = 688,000
Scope 3 – Downstream transportation: 85,000 + 57,000 = 142,000
Scope 3 – Use of sold products: 4,103,000 + 3,753,000 = 7,856,000
Scope 3 – End-of-life of sold products: 31,000 + 18,000 = 59,000
Scope 3 – Downstream leased assets: 700 + 800 = 1,500

Total Scope 3 emissions = 1,200,000 + 199,000 + 150,000 + 1,800 + 880,807 + 688,000 + 142,000 + 7,856,000 + 59,0

In [ ]:
result = qa_chain(" what isTotal hazardous waste  produced in tons in FY20? ")

 Based on the information provided, I can answer your question.

The total hazardous waste produced in tons in FY20 was 7,280 tons. This information can be found in the second line of the text under the heading "Recycled Hazardous Waste".


In [ ]:
result = qa_chain("What Total amount of   waste of produced in tons  in 2020 ? ")

 Based on the information provided, I can see that the total amount of waste produced in tons in 2020 is not explicitly stated. However, I can calculate it based on the information given.

From the table, we have:

* Recycled hazardous waste: 2,122 + 1,376 = 3,500 tons
* Solid waste diverted: 60,000 tons

So, the total amount of waste produced in tons in 2020 would be:

3,500 + 60,000 = 63,500 tons

Therefore, the answer is approximately 63,500 tons.


In [ ]:
result = qa_chain(" what is  water withdrawals  FY20? ")

 Based on the information provided in the table, the water withdrawals for FY20 are 7,618,089 cubic meters.


In [ ]:
result = qa_chain(" what isTotal number of  Employees  in 2020 ? ")

 Based on the information provided in the fact sheet, there is no direct mention of the total number of employees in 2020. However, we can infer that the company had at least 1,602 employees in 2020 based on the landfilled waste category, which states that 1,602 metric tons of waste were landfilled in 2020. This is because the company reports waste data in metric tons, and each ton of waste is equivalent to 1,000 kilograms or 100,000 grams. Therefore, if 1,602 metric tons of waste were landfilled in 2020, then the company must have had at least 1,602 employees in 2020.

However, it's important to note that this is a rough estimate and may not be entirely accurate. The company may have had more or fewer employees in 2020, depending on various factors such as employee turnover, new hires, and layoffs.


#Bayer

In [ ]:

result = qa_chain("What Scope 1 in 2020 Greenhouse Gas Emissions (Scope 1 and 2)table ethod metric tons CO2e?")



 Based on the information provided in the table, the Scope 1 emissions in 2020 were 2.01 million metric tons CO2e.


In [ ]:



result = qa_chain("What is Scope 2: Indirect emissions2 according to the market-based method emissions metric tons CO2e in 2020  ?")



 Based on the information provided, Scope 2: Indirect emissions2 according to the market-based method emissions in 2020 were 1.57 million metric tons CO2e.


In [ ]:



result = qa_chain("What Scope 2 market-based method  emissions metric tons CO2e in 2020")



 Based on the information provided, the Scope 2 market-based method emissions in 2020 were 1.57 million metric tons CO2e.


In [ ]:



result = qa_chain("What Scope 3 Indirect emissions from our upstream and downstream value chain (by materiality)1 table metric tons CO2e in 2020 ?"
)



 Based on the information provided, the Scope 3 indirect emissions from the upstream and downstream value chain (by materiality) in 2020 were 8.87 million metric tons CO2e.


In [ ]:



result = qa_chain("What Scope 3  metric tons CO2e  ?"
)



 Based on the information provided, I can answer the following question:

What is the Scope 3 indirect emissions from upstream and downstream value chain to attain the SBT?

According to the information provided, the Scope 3 indirect emissions from upstream and downstream value chain to attain the SBT are 8.87 million metric tons CO2e in 2019, representing 88% of the company's total Scope 3 emissions.


In [ ]:
result = qa_chain("What is volume of waste   produced in 1,000 metric tons  ? ")



 Based on the information provided, I cannot determine the exact volume of waste produced in 1,000 metric tons. However, I can provide some relevant information:

* The total volume of waste disposed of in 2019 and 2020 was 872,943 and 943, respectively.
* The proportion of hazardous waste that was recycled was 12.4% in both years.
* The volume of hazardous waste disposed of to landfill fell in 2020 due to the completion of construction work at the Vapi site in India.

Without more specific information about the type and amount of waste produced in 1,000 metric tons, I cannot accurately calculate the volume of waste produced.


In [ ]:






result = qa_chain(" what is Hazardous waste disposed of produced in  2020 tons  ? ")

 Based on the information provided, the answer to your question is:

Hazardous waste disposed of produced in 2020 was 305 tons.


In [ ]:



result = qa_chain(" what is  Total water usage in cubic meter  in 2020 ? ")








 Based on the information provided, the total water usage in cubic meters in 2020 is 57 million cubic meters. This is calculated by adding the total water sourced (59 million cubic meters) and the volume of treated production wastewater (25 million cubic meters), and then subtracting the volume of water recycled (380 million cubic meters).

59 million cubic meters + 25 million cubic meters - 380 million cubic meters = 57 million cubic meters


In [ ]:
result = qa_chain(" what isTotal number of  Employees in 2020 ? ")

 Based on the information provided, the total number of employees in 2020 was 99,538.


#Philip Morris International Inc.

In [ ]:
result = qa_chain("What scope 1 (metric tons)  climate protection table 2020?")



 Based on the information provided in the table, the scope 1 (metric tons) for climate protection in 2020 is 322,633.


In [ ]:
result = qa_chain("What scope2 (metric tons)  climate protection table 2020?")



 Based on the information provided in the climate protection table, the scope 2 (metric tons) for 2020 is 158,672.


In [ ]:
result = qa_chain("What scope 2 (metric tons) market- based climate protection table 2020?")



 Based on the information provided in the table, the scope 2 (metric tons) market-based climate protection for 2020 is 90,366.


In [ ]:
result = qa_chain("What scope 2 (metric tons) location- based climate protection table 2020?")



 Based on the information provided in the table, I can see that there is no data for scope 2 (metric tons) in the location-based climate protection table for 2020. The table only provides data for scope 1 and scope 3 emissions. Therefore, I cannot answer the question as there is no relevant information available.


In [ ]:
result = qa_chain("What scope 3 (metric tons) climate protection table 2020?")





 Based on the information provided in the table, the scope 3 (metric tons) for climate protection in 2020 is 3,590.


In [ ]:
result = qa_chain("What Total amount of   waste of produced in tons  in 2020 ? ")



/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Based on the information provided, I can answer the following questions:

What was the total amount of waste generated in 2020?

According to the table, the total amount of waste generated in 2020 was 119,750 metric tons.


In [ ]:
result = qa_chain(" what isTotal hazardous waste  produced in tons  in 2020 ? ")



 Based on the information provided, I cannot determine the total amount of hazardous waste produced in 2020. The information only provides data on the proportion of hazardous waste that was recycled, incinerated with energy recovery, and disposed to landfill, but it does not provide the total amount of hazardous waste generated. Additionally, the information only covers up to 2019, and there is no data available for 2020. Therefore, I cannot answer the question.


In [ ]:
result = qa_chain(" what is  Total water usage in cubic meter in circular environment table in 2020 ? ")



 Based on the information provided in the text, the total water usage in cubic meters in the circular environment table in 2020 is 1,299,000 m3.


In [ ]:
result = qa_chain(" what isTotal number of  Employees n 2020 ? ")



 Based on the information provided, the total number of employees of Philip Morris International as of year-end 2020 is:

Permanent employees:

* Women: 28,401
* Men: 39,835

Temporary employees:

* Women: 1,109
* Men: 1,504

Total number of employees:

28,401 + 39,835 + 1,109 + 1,504 = 79,445

Therefore, the total number of employees of Philip Morris International as of year-end 2020 was approximately 79,445.


#Nokia

In [ ]:
result = qa_chain("What Scope 1GHG Emissions tons CO2e in 2020")



 Based on the information provided in the Nokia People & Planet Report 2020, the Scope 1 GHG emissions in 2020 were 116,300 metric tons CO2e.


In [ ]:
result = qa_chain("What Scope 2 GHG emissions, Market-based tons CO2e in 2020")



 Based on the information provided in the Nokia People & Planet Report 2020, the Scope 2 GHG emissions, Market-based in 2020 were 327,200 tons CO2e.


In [ ]:
result = qa_chain("What Scope 2GHG Emissions ,location- based in 2020 ?")



 Based on the information provided in the Nokia People & Planet Report 2020, the Scope 2 GHG emissions, location-based in 2020 were 327,200 metric tons CO2e.


In [ ]:
result = qa_chain("What Scope 3 in tons in 2020")



 Sure! Here is the answer to your question based on the provided text:

Based on the information provided in the text, the Scope 3 GHG emissions related to upstream transportation and distribution of products in 2020 were approximately 255,200 metric tons CO2e.


In [ ]:
result = qa_chain("What Total  waste  ")



 Based on the information provided, I cannot determine the total waste generated by Nokia in 2020. The information provided only includes partial data on waste generation, and there is no total waste figure given. Additionally, the data provided is not comprehensive, as it does not include all types of waste, such as packaging waste and equipment collected at the end-of-life. Without complete and accurate data, it is not possible to calculate a total waste figure for Nokia in 2020.


In [ ]:
result = qa_chain(" what isTotal hazardous waste  in circular economy in 2020? ")



 Based on the information provided in the text, I cannot determine the total amount of hazardous waste generated by Nokia in 2020. The text provides information on the different types of hazardous waste, but it does not provide a total amount for all of these wastes combined. Therefore, I do not know the answer to your question.


In [ ]:
result = qa_chain(" what is  Total water usage in cubic meter in circular environment table in 2020 ? ")



 Based on the information provided in the text, the total water usage in cubic meters in the circular environment table in 2020 is 1,299,000 m3.


In [ ]:
result = qa_chain(" what isTotal number of  Employees year-end situation 2020 ? ")



 Based on the information provided in the text, the total number of employees year-end situation in 2020 was 89,978. This can be found in the "People data" section of the report, under the heading "Total number of employees."
